In [1]:
# !python3 -m pip install -U "autogen-agentchat" "autogen-ext[openai,azure]"

# Simplest Example using a Fake Tool

In [3]:
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.ui import Console
from autogen_ext.models.openai import OpenAIChatCompletionClient


# Define a tool
async def get_weather(city: str) -> str:
    return f"The weather in {city} is 73 degrees and Sunny."


In [5]:
import os
OPENAI_KEY = os.environ.get('OPENAI_API_KEY')

In [6]:
async def main() -> None:
    # Define an agent
    weather_agent = AssistantAgent(
        name="weather_agent",
        model_client=OpenAIChatCompletionClient(
            model="gpt-4o",
            api_key=OPENAI_KEY,
        ),
        tools=[get_weather],
    )

    # Define a team with a single agent and maximum auto-gen turns of 1.
    agent_team = RoundRobinGroupChat([weather_agent], max_turns=1)

    while True:
        # Get user input from the console.
        user_input = input("Enter a message (type 'exit' to leave): ")
        if user_input.strip().lower() == "exit":
            break
        # Run the team and stream messages to the console.
        stream = agent_team.run_stream(task=user_input)
        await Console(stream)


# NOTE: if running this inside a Python script you'll need to use asyncio.run(main()).
await main()

Enter a message (type 'exit' to leave):  weather in SF


---------- user ----------
weather in SF
---------- weather_agent ----------
[FunctionCall(id='call_f3WkHWqXlC7hQocfXGvINJYw', arguments='{"city":"SF"}', name='get_weather')]
---------- weather_agent ----------
[FunctionExecutionResult(content='The weather in SF is 73 degrees and Sunny.', call_id='call_f3WkHWqXlC7hQocfXGvINJYw')]
---------- weather_agent ----------
The weather in SF is 73 degrees and Sunny.


Enter a message (type 'exit' to leave):  how big is the moon


---------- user ----------
how big is the moon
---------- weather_agent ----------
The Moon has a diameter of about 3,474 kilometers (2,159 miles) and a surface area of approximately 37.9 million square kilometers (14.6 million square miles). It is roughly one-quarter the diameter of Earth. TERMINATE


Enter a message (type 'exit' to leave):  what time is it?


---------- user ----------
what time is it?
---------- weather_agent ----------
I'm sorry, but I don't have the capability to provide real-time information such as the current time. You might want to check your device's clock or a reliable source for the most accurate time. TERMINATE


Enter a message (type 'exit' to leave):  exit


# A more complicated multi-agent platform (still with fake tools :) )

https://microsoft.github.io/autogen/stable//user-guide/agentchat-user-guide/swarm.html#stock-research-example

Here we have four agents:

- Planner: The central coordinator that delegates specific tasks to specialized agents based on their expertise. The planner ensures that each agent is utilized efficiently and oversees the overall workflow.

- Financial Analyst: A specialized agent responsible for analyzing financial metrics and stock data using tools such as get_stock_data.

- News Analyst: An agent focused on gathering and summarizing recent news articles relevant to the stock, using tools such as get_news.

- Writer: An agent tasked with compiling the findings from the stock and news analysis into a cohesive final report.

In [11]:
from typing import Dict, Any, List

async def get_stock_data(symbol: str) -> Dict[str, Any]:
    """Get stock market data for a given symbol"""
    return {"price": 180.25, "volume": 1000000, "pe_ratio": 65.4, "market_cap": "700B"}


async def get_news(query: str) -> List[Dict[str, str]]:
    """Get recent news articles about a company"""
    return [
        {
            "title": "Tesla Expands Cybertruck Production",
            "date": "2024-03-20",
            "summary": "Tesla ramps up Cybertruck manufacturing capacity at Gigafactory Texas, aiming to meet strong demand.",
        },
        {
            "title": "Tesla FSD Beta Shows Promise",
            "date": "2024-03-19",
            "summary": "Latest Full Self-Driving beta demonstrates significant improvements in urban navigation and safety features.",
        },
        {
            "title": "Model Y Dominates Global EV Sales",
            "date": "2024-03-18",
            "summary": "Tesla's Model Y becomes best-selling electric vehicle worldwide, capturing significant market share.",
        },
    ]

In [12]:
model_client = OpenAIChatCompletionClient(
    model="gpt-4o",
    api_key=OPENAI_KEY,
)

planner = AssistantAgent(
    "planner",
    model_client=model_client,
    handoffs=["financial_analyst", "news_analyst", "writer"],
    system_message="""You are a research planning coordinator.
    Coordinate market research by delegating to specialized agents:
    - Financial Analyst: For stock data analysis
    - News Analyst: For news gathering and analysis
    - Writer: For compiling final report
    Always send your plan first, then handoff to appropriate agent.
    Always handoff to a single agent at a time.
    Use TERMINATE when research is complete.""",
)

financial_analyst = AssistantAgent(
    "financial_analyst",
    model_client=model_client,
    handoffs=["planner"],
    tools=[get_stock_data],
    system_message="""You are a financial analyst.
    Analyze stock market data using the get_stock_data tool.
    Provide insights on financial metrics.
    Always handoff back to planner when analysis is complete.""",
)

news_analyst = AssistantAgent(
    "news_analyst",
    model_client=model_client,
    handoffs=["planner"],
    tools=[get_news],
    system_message="""You are a news analyst.
    Gather and analyze relevant news using the get_news tool.
    Summarize key market insights from news.
    Always handoff back to planner when analysis is complete.""",
)

writer = AssistantAgent(
    "writer",
    model_client=model_client,
    handoffs=["planner"],
    system_message="""You are a financial report writer.
    Compile research findings into clear, concise reports.
    Always handoff back to planner when writing is complete.""",
)

In [17]:
from autogen_agentchat.conditions import HandoffTermination, TextMentionTermination
from autogen_agentchat.teams import Swarm

# Define termination condition
termination = TextMentionTermination("TERMINATE")

research_team = Swarm(
    participants=[planner, financial_analyst, news_analyst, writer], termination_condition=termination
)

task = "Conduct market research for TSLA stock"
result = await Console(research_team.run_stream(task=task))

---------- user ----------
Conduct market research for TSLA stock
---------- planner ----------
[FunctionCall(id='call_jDqCj22EVP5FICNHGQ0EtXwT', arguments='{}', name='transfer_to_financial_analyst'), FunctionCall(id='call_z8BePI6mSkrejThvbykx2nJJ', arguments='{}', name='transfer_to_news_analyst')]
---------- planner ----------
[FunctionExecutionResult(content='Transferred to financial_analyst, adopting the role of financial_analyst immediately.', call_id='call_jDqCj22EVP5FICNHGQ0EtXwT'), FunctionExecutionResult(content='Transferred to news_analyst, adopting the role of news_analyst immediately.', call_id='call_z8BePI6mSkrejThvbykx2nJJ')]
---------- planner ----------
Transferred to financial_analyst, adopting the role of financial_analyst immediately.


/Users/sinanozdemir/Teaching/Pearson/oreilly-ai-agents/.venv/lib/python3.11/site-packages/autogen_agentchat/teams/_group_chat/_chat_agent_container.py:53: UserWarning: Multiple handoffs detected only the first is executed: ['transfer_to_financial_analyst', 'transfer_to_news_analyst']. Disable parallel tool call in the model client to avoid this warning.
  async for msg in self._agent.on_messages_stream(self._message_buffer, ctx.cancellation_token):


---------- financial_analyst ----------
[FunctionCall(id='call_7OlkwXfoT2AwZUl1cacWIAkK', arguments='{"symbol":"TSLA"}', name='get_stock_data')]
---------- financial_analyst ----------
[FunctionExecutionResult(content="{'price': 180.25, 'volume': 1000000, 'pe_ratio': 65.4, 'market_cap': '700B'}", call_id='call_7OlkwXfoT2AwZUl1cacWIAkK')]
---------- financial_analyst ----------
{'price': 180.25, 'volume': 1000000, 'pe_ratio': 65.4, 'market_cap': '700B'}
---------- financial_analyst ----------
The current data for Tesla (TSLA) stock is:

- **Stock Price**: $180.25
- **Trading Volume**: 1,000,000 shares
- **Price-to-Earnings Ratio (P/E)**: 65.4
- **Market Capitalization**: $700 billion

### Analysis:

1. **Stock Price**: The current price of Tesla shares sits at $180.25, offering insight into the market's valuation of the company at this moment.

2. **Volume**: With a trading volume of 1,000,000 shares, there is noteworthy interest and liquidity, allowing transactions to occur smoothly wi

In [24]:
for message in result.messages:
    print(f'Source: {message.source}')
    print(f'models_usage: {message.models_usage}')
    print(f'content: {message.content}')
    print('-------')

Source: user
models_usage: None
content: Conduct market research for TSLA stock
-------
Source: planner
models_usage: RequestUsage(prompt_tokens=1454, completion_tokens=49)
content: [FunctionCall(id='call_jDqCj22EVP5FICNHGQ0EtXwT', arguments='{}', name='transfer_to_financial_analyst'), FunctionCall(id='call_z8BePI6mSkrejThvbykx2nJJ', arguments='{}', name='transfer_to_news_analyst')]
-------
Source: planner
models_usage: None
content: [FunctionExecutionResult(content='Transferred to financial_analyst, adopting the role of financial_analyst immediately.', call_id='call_jDqCj22EVP5FICNHGQ0EtXwT'), FunctionExecutionResult(content='Transferred to news_analyst, adopting the role of news_analyst immediately.', call_id='call_z8BePI6mSkrejThvbykx2nJJ')]
-------
Source: planner
models_usage: None
content: Transferred to financial_analyst, adopting the role of financial_analyst immediately.
-------
Source: financial_analyst
models_usage: RequestUsage(prompt_tokens=571, completion_tokens=17)
conte

In [25]:
async def main() -> None:
   
    while True:
        # Get user input from the console.
        user_input = input("Enter a message (type 'exit' to leave): ")
        if user_input.strip().lower() == "exit":
            break
        # Run the team and stream messages to the console.
        stream = research_team.run_stream(task=user_input)
        await Console(stream)


# NOTE: if running this inside a Python script you'll need to use asyncio.run(main()).
await main()

Enter a message (type 'exit' to leave):  hi how are you?


---------- user ----------
hi how are you?
---------- planner ----------
Hello! I'm here to help with your market research needs. How can I assist you today?
---------- planner ----------
TERMINATE


Enter a message (type 'exit' to leave):  can you do some basic research on tsla


---------- user ----------
can you do some basic research on tsla
---------- planner ----------
[FunctionCall(id='call_YrOqnWgn4l4519vJpSCHZOlY', arguments='{}', name='transfer_to_financial_analyst'), FunctionCall(id='call_VxPwosUNCgX9RCJhlt7L4wkK', arguments='{}', name='transfer_to_news_analyst')]
---------- planner ----------
[FunctionExecutionResult(content='Transferred to financial_analyst, adopting the role of financial_analyst immediately.', call_id='call_YrOqnWgn4l4519vJpSCHZOlY'), FunctionExecutionResult(content='Transferred to news_analyst, adopting the role of news_analyst immediately.', call_id='call_VxPwosUNCgX9RCJhlt7L4wkK')]
---------- planner ----------
Transferred to financial_analyst, adopting the role of financial_analyst immediately.
---------- financial_analyst ----------
[FunctionCall(id='call_XkCQMHM0S71sa6IlTDRoSddn', arguments='{"symbol":"TSLA"}', name='get_stock_data')]
---------- financial_analyst ----------
[FunctionExecutionResult(content="{'price': 180.25, 

Enter a message (type 'exit' to leave):  exit
